In [1]:
# === SEKCJA 1: IMPORT I WCZYTANIE DANYCH ===
import pandas as pd
import numpy as np
import re
import os
import pickle
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense, Dropout, BatchNormalization, LeakyReLU, Reshape, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Ignorowanie specyficznych ostrzeżeń
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
# Wstępna konfiguracja GPU, jeśli jest dostępne
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print("--- Wczytywanie danych ---\n")
try:
    df_main_raw = pd.read_csv('saleflats_mazowieckie_c.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
    df_slownik = pd.read_csv('slownik_finalny_z_hierarchia.csv', sep=';')
    print("Pliki wczytane pomyślnie.")
except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku: {e.filename}.")
    raise

# --- Przygotowanie danych (identyczne jak w Twoim notatniku) ---
print("\n--- Przygotowanie danych do modelu ---")
df_main = df_main_raw.copy()
df_main.columns = [i for i in range(53)] + ['WojewodztwoID', 'PowiatID', 'GminaID', 'RodzajGminyID', 'MiastoID', 'DzielnicaID', 'UlicaID']
main_cols_map = {0: 'SaleId', 4: 'Description', 5: 'Area', 6: 'Price', 17: 'NumberOfRooms', 35: 'Floor', 36: 'Floors'}
df_main.rename(columns=main_cols_map, inplace=True)

numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors']
id_features = ['UlicaID']
for col in numeric_features + id_features:
    df_main[col] = pd.to_numeric(df_main[col], errors='coerce')
df_main.dropna(subset=['Description'] + numeric_features + id_features, inplace=True)
df_main['UlicaID'] = df_main['UlicaID'].astype(int)

df_merged = pd.merge(df_main, df_slownik, on='UlicaID', how='inner')
print(f"Liczba ofert po połączeniu ze słownikiem: {len(df_merged)}")

df_model_ready = df_merged.copy()
df_model_ready['description_enriched'] = (df_model_ready['Dzielnica_Name'] + " " + df_model_ready['Ulica_Name'] + " ") * 5 + df_model_ready['Description']
def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())
df_model_ready['description_clean'] = df_model_ready['description_enriched'].apply(clean_text)

MAX_WORDS, MAX_LEN = 20000, 250
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_model_ready['description_clean'])
X_text = pad_sequences(tokenizer.texts_to_sequences(df_model_ready['description_clean']), maxlen=MAX_LEN)

df_model_ready['Price_per_sqm'] = df_model_ready['Price'] / df_model_ready['Area']
df_model_ready['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)
numeric_features_cols = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'Price_per_sqm']
numeric_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
X_numeric = numeric_pipeline.fit_transform(df_model_ready[numeric_features_cols])

le_dzielnica = LabelEncoder()
y_dzielnica = le_dzielnica.fit_transform(df_model_ready['Dzielnica_Name'])
num_classes_dzielnica = len(le_dzielnica.classes_)
le_ulica = LabelEncoder()
y_ulica = le_ulica.fit_transform(df_model_ready['Ulica_Name'])
num_classes_ulica = len(le_ulica.classes_)

print(f"\nProblem przygotowany do modelowania:")
print(f" - Liczba klas (dzielnice): {num_classes_dzielnica}")
print(f" - Liczba klas (ulice): {num_classes_ulica}")

# Podział na zbiory TRENINGOWY i WALIDACYJNY
# To jest KLUCZOWE: zbiór walidacyjny pozostanie NIETKNIĘTY. Augmentować będziemy TYLKO zbiór treningowy.
train_indices, val_indices = train_test_split(range(len(df_model_ready)), test_size=0.2, random_state=42, stratify=y_dzielnica)

X_train_text, X_val_text = X_text[train_indices], X_text[val_indices]
X_train_num, X_val_num = X_numeric[train_indices], X_numeric[val_indices]
y_train_dzielnica, y_val_dzielnica = y_dzielnica[train_indices], y_dzielnica[val_indices]
y_train_ulica, y_val_ulica = y_ulica[train_indices], y_ulica[val_indices]

print("\nDane podzielone na zbiory treningowe i walidacyjne.")
print(f"Rozmiar zbioru treningowego: {len(X_train_text)}")
print(f"Rozmiar zbioru walidacyjnego: {len(X_val_text)}")

--- Wczytywanie danych ---

Pliki wczytane pomyślnie.

--- Przygotowanie danych do modelu ---
Liczba ofert po połączeniu ze słownikiem: 9548

Problem przygotowany do modelowania:
 - Liczba klas (dzielnice): 18
 - Liczba klas (ulice): 703

Dane podzielone na zbiory treningowe i walidacyjne.
Rozmiar zbioru treningowego: 7638
Rozmiar zbioru walidacyjnego: 1910


C:\Users\Ai\AppData\Local\Temp\ipykernel_5388\3355088492.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model_ready['Price_per_sqm'].replace([np.inf, -np.inf], np.nan, inplace=True)


In [2]:
# === SEKCJA 2: BUDOWA I KOMPILACJA MODELU Z PRIORYTETEM DLA TEKSTU ===

def build_priority_classifier(num_classes_d, num_classes_u, max_words, max_len, num_numeric_features):
    """Definiuje architekturę modelu z trzema głowami wyjściowymi."""
    # --- Wejścia ---
    input_text = Input(shape=(max_len,), name='text_input')
    input_numeric = Input(shape=(num_numeric_features,), name='numeric_input')
    
    # --- Trzon tekstowy ---
    text_embedding = Embedding(input_dim=max_words, output_dim=128)(input_text)
    lstm_out = LSTM(128, dropout=0.3, name='lstm_layer')(text_embedding)
    
    # --- Gałąź wyjściowa TYLKO dla tekstu (dla dzielnicy) ---
    text_only_branch = Dense(64, activation='relu')(lstm_out)
    text_only_branch = Dropout(0.5)(text_only_branch)
    output_dzielnica_text = Dense(num_classes_d, activation='softmax', name='output_dzielnica_text')(text_only_branch)

    # --- Gałąź połączona (tekst + dane numeryczne) ---
    concatenated = Concatenate()([lstm_out, input_numeric])
    common_dense = Dense(128, activation='relu')(concatenated)
    common_dense = Dropout(0.5)(common_dense)

    # Wyjście dla dzielnicy z danych połączonych
    dzielnica_combined_branch = Dense(64, activation='relu')(common_dense)
    output_dzielnica_combined = Dense(num_classes_d, activation='softmax', name='output_dzielnica_combined')(dzielnica_combined_branch)
    
    # Wyjście dla ulicy z danych połączonych
    ulica_combined_branch = Dense(256, activation='relu')(common_dense)
    output_ulica_combined = Dense(num_classes_u, activation='softmax', name='output_ulica_combined')(ulica_combined_branch)
    
    # --- Model z trzema wyjściami ---
    model = Model(
        inputs=[input_text, input_numeric], 
        outputs=[output_dzielnica_text, output_dzielnica_combined, output_ulica_combined]
    )
    
    # --- Definicja strat i wag ---
    losses = {
        "output_dzielnica_text": "sparse_categorical_crossentropy",
        "output_dzielnica_combined": "sparse_categorical_crossentropy",
        "output_ulica_combined": "sparse_categorical_crossentropy",
    }
    
    loss_weights = {
        "output_dzielnica_text": 2.0,      # <-- Błąd tutaj jest karany 2x mocniej!
        "output_dzielnica_combined": 1.0,  # <-- Standardowa waga
        "output_ulica_combined": 0.5       # <-- Mniejsza waga, bo to trudniejsze zadanie
    }
    
    metrics = {
        "output_dzielnica_text": "accuracy",
        "output_dzielnica_combined": "accuracy",
        "output_ulica_combined": "accuracy"
    }
    
    model.compile(
        optimizer='adam',
        loss=losses,
        loss_weights=loss_weights,
        metrics=metrics
    )
    return model

print("\n--- Budowa i kompilacja modelu z priorytetem dla tekstu ---")
# Tworzymy instancję nowego modelu
priority_model = build_priority_classifier(
    num_classes_d=num_classes_dzielnica,
    num_classes_u=num_classes_ulica,
    max_words=MAX_WORDS,
    max_len=MAX_LEN,
    num_numeric_features=X_train_num.shape[1]
)
priority_model.summary()


--- Budowa i kompilacja modelu z priorytetem dla tekstu ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 250)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 250, 128)          │       2,560,000 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_layer (LSTM)             │ (None, 128)               │         131,584 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ numeric_input (InputLayer)    │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 134)               │               0 │ lstm_layer[0][0],          │
│                               │                           │                 │ numeric_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 128)               │          17,280 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 64)                │           8,256 │ lstm_layer[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 64)                │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 64)                │           8,256 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 256)               │          33,024 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dzielnica_text (Dense) │ (None, 18)                │           1,170 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dzielnica_combined     │ (None, 18)                │           1,170 │ dense_2[0][0]              │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_ulica_combined (Dense) │ (None, 703)               │         180,671 │ dense_3[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,941,411 (11.22 MB)

 Trainable params: 2,941,411 (11.22 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# === SEKCJA 3: TRENING MODELU Z PRIORYTETEM ---

# Przygotowanie danych wejściowych i wyjściowych
X_train = [X_train_text, X_train_num]
y_train = {
    'output_dzielnica_text': y_train_dzielnica,
    'output_dzielnica_combined': y_train_dzielnica,
    'output_ulica_combined': y_train_ulica
}

X_val = [X_val_text, X_val_num]
y_val = {
    'output_dzielnica_text': y_val_dzielnica,
    'output_dzielnica_combined': y_val_dzielnica,
    'output_ulica_combined': y_val_ulica
}

# Definicja callbacków
callbacks = [
    EarlyStopping(monitor='val_output_ulica_combined_accuracy', patience=5, restore_best_weights=True, verbose=1, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)
]

print("\n--- Rozpoczynam trening modelu z priorytetem ---")
history_priority = priority_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)


--- Rozpoczynam trening modelu z priorytetem ---
Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 27s 395ms/step - loss: 10.6614 - output_dzielnica_combined_accuracy: 0.2416 - output_dzielnica_combined_loss: 2.5224 - output_dzielnica_text_accuracy: 0.2572 - output_dzielnica_text_loss: 2.5977 - output_ulica_combined_accuracy: 0.1259 - output_ulica_combined_loss: 5.8869 - val_loss: 8.1940 - val_output_dzielnica_combined_accuracy: 0.4178 - val_output_dzielnica_combined_loss: 1.8276 - val_output_dzielnica_text_accuracy: 0.3230 - val_output_dzielnica_text_loss: 2.0074 - val_output_ulica_combined_accuracy: 0.1832 - val_output_ulica_combined_loss: 4.7018 - learning_rate: 0.0010
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 22s 368ms/step - loss: 8.1620 - output_dzielnica_combined_accuracy: 0.4236 - output_dzielnica_combined_loss: 1.8285 - output_dzielnica_text_accuracy: 0.3283 - output_dzielnica_text_loss: 2.0266 - output_ulica_combined_accuracy: 0.2049 - output_ulica_combined_loss: 4.5607 - val_loss: 7.8494 -

In [4]:
# === SEKCJA 4: OCENA I ZAPIS MODELU ---

print("\n--- Trening zakończony. Ocena modelu ---")
results = priority_model.evaluate(X_val, y_val, verbose=0)

# Wyświetlanie wyników
print("\nWyniki na zbiorze walidacyjnym:")
print(f" - Całkowita strata: {results[0]:.4f}")
print(f" - Dokładność (dzielnica z tekstu):     {results[4]:.4f}")
print(f" - Dokładność (dzielnica połączona):    {results[5]:.4f}")
print(f" - Dokładność (ulica połączona):        {results[6]:.4f} <--- Kluczowy wynik")

# --- Zapis artefaktów do produkcji ---
artifacts_dir_priority = 'model_artifacts_priority'
os.makedirs(artifacts_dir_priority, exist_ok=True)

# 1. Zapis modelu
priority_model.save(os.path.join(artifacts_dir_priority, 'priority_model.keras'))

# 2. Zapis pozostałych artefaktów (są takie same jak wcześniej)
with open(os.path.join(artifacts_dir_priority, 'tokenizer.pkl'), 'wb') as f: pickle.dump(tokenizer, f)
with open(os.path.join(artifacts_dir_priority, 'numeric_pipeline.pkl'), 'wb') as f: pickle.dump(numeric_pipeline, f)
with open(os.path.join(artifacts_dir_priority, 'le_dzielnica.pkl'), 'wb') as f: pickle.dump(le_dzielnica, f)
with open(os.path.join(artifacts_dir_priority, 'le_ulica.pkl'), 'wb') as f: pickle.dump(le_ulica, f)
    
print(f"\nWszystkie artefakty dla nowego modelu zostały zapisane w folderze: '{artifacts_dir_priority}'")


--- Trening zakończony. Ocena modelu ---

Wyniki na zbiorze walidacyjnym:
 - Całkowita strata: 4.9658
 - Dokładność (dzielnica z tekstu):     0.7387
 - Dokładność (dzielnica połączona):    0.7152
 - Dokładność (ulica połączona):        0.3021 <--- Kluczowy wynik

Wszystkie artefakty dla nowego modelu zostały zapisane w folderze: 'model_artifacts_priority'


In [5]:
# === SEKCJA 5: TESTOWANIE NOWEGO MODELU NA PROBLEMATYCZNYM PRZYKŁADZIE ===

# ========= POPRAWKA: Dodajemy brakujące importy =========
from tensorflow.keras.models import load_model
import json
import re # re jest używane w funkcji clean_text
# =========================================================

# --- Wczytanie artefaktów (dla pewności, że używamy zapisanych) ---
artifacts_dir_priority = 'model_artifacts_priority'
try:
    loaded_priority_model = load_model(os.path.join(artifacts_dir_priority, 'priority_model.keras'))
    with open(os.path.join(artifacts_dir_priority, 'tokenizer.pkl'), 'rb') as f: loaded_tokenizer = pickle.load(f)
    with open(os.path.join(artifacts_dir_priority, 'numeric_pipeline.pkl'), 'rb') as f: loaded_pipeline = pickle.load(f)
    with open(os.path.join(artifacts_dir_priority, 'le_dzielnica.pkl'), 'rb') as f: loaded_le_d = pickle.load(f)
    with open(os.path.join(artifacts_dir_priority, 'le_ulica.pkl'), 'rb') as f: loaded_le_u = pickle.load(f)
    print("\nArtefakty dla modelu z priorytetem wczytane pomyślnie.")
except Exception as e:
    print(f"Błąd wczytywania, używam zmiennych z pamięci: {e}")
    # W razie problemu użyj zmiennych z pamięci
    loaded_priority_model, loaded_tokenizer, loaded_pipeline, loaded_le_d, loaded_le_u = \
    priority_model, tokenizer, numeric_pipeline, le_dzielnica, le_ulica


# --- Zmodyfikowana funkcja predykcyjna ---
def predict_location_priority(json_data, model, tokenizer, pipeline, le_d, le_u):
    data = json.loads(json_data)
    
    def clean_text(text): return re.sub(r'[^a-ząęółśżźćń ]', '', str(text).lower())
    clean_desc = clean_text(data['Description'])
    text_sequence = tokenizer.texts_to_sequences([clean_desc])
    padded_text = pad_sequences(text_sequence, maxlen=250)

    price_per_sqm = data['Price'] / data['Area'] if data['Area'] > 0 else 0
    numeric_values = np.array([data['Area'], data['Price'], data['NumberOfRooms'], data['Floor'], data['Floors'], price_per_sqm]).reshape(1, -1)
    scaled_numeric = pipeline.transform(numeric_values)
    
    predictions = model.predict([padded_text, scaled_numeric], verbose=0)
    pred_dzielnica_text_probs, pred_dzielnica_combined_probs, pred_ulica_combined_probs = predictions
    
    pred_dzielnica_combined_index = np.argmax(pred_dzielnica_combined_probs, axis=1)[0]
    pred_ulica_combined_index = np.argmax(pred_ulica_combined_probs, axis=1)[0]
    pred_dzielnica_text_index = np.argmax(pred_dzielnica_text_probs, axis=1)[0]
    
    return {
        "predicted_district_text_only": le_d.inverse_transform([pred_dzielnica_text_index])[0],
        "district_text_confidence": f"{pred_dzielnica_text_probs[0][pred_dzielnica_text_index]:.2%}",
        "predicted_district_combined": le_d.inverse_transform([pred_dzielnica_combined_index])[0],
        "district_combined_confidence": f"{pred_dzielnica_combined_probs[0][pred_dzielnica_combined_index]:.2%}",
        "predicted_street_combined": le_u.inverse_transform([pred_ulica_combined_index])[0],
        "street_combined_confidence": f"{pred_ulica_combined_probs[0][pred_ulica_combined_index]:.2%}"
    }

# --- Uruchomienie predykcji ---
json_input_data = """
{
  "Description": "Sprzedam słoneczne, trzypokojowe mieszkanie na Mokotowie. Doskonała lokalizacja przy ulicy Puławskiej zapewnia świetną komunikację z centrum. W pobliżu metro Racławicka. Mieszkanie w zadbanej kamienicy z cegły.",
  "Area": 68.0,
  "Price": 980000,
  "NumberOfRooms": 3,
  "Floor": 3,
  "Floors": 5
}
"""

print("\n\n--- TESTOWANIE NOWEGO MODELU NA PROBLEMATYCZNYM PRZYKŁADZIE ---")
final_prediction = predict_location_priority(
    json_input_data, 
    loaded_priority_model, 
    loaded_tokenizer, 
    loaded_pipeline, 
    loaded_le_d, 
    loaded_le_u
)

print("\n--- WYNIK PREDYKCJI (model z priorytetem) ---")
print(f"Predykcja samej gałęzi tekstowej: {final_prediction['predicted_district_text_only']} (Pewność: {final_prediction['district_text_confidence']})")
print("--------------------------------------------------")
print(f"Finalna predykcja Dzielnicy: {final_prediction['predicted_district_combined']} (Pewność: {final_prediction['district_combined_confidence']})")
print(f"Finalna predykcja Ulicy:     {final_prediction['predicted_street_combined']} (Pewność: {final_prediction['street_combined_confidence']})")


Artefakty dla modelu z priorytetem wczytane pomyślnie.


--- TESTOWANIE NOWEGO MODELU NA PROBLEMATYCZNYM PRZYKŁADZIE ---


C:\Users\Ai\AppData\Local\miniconda3\envs\projekt1\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(



--- WYNIK PREDYKCJI (model z priorytetem) ---
Predykcja samej gałęzi tekstowej: Wawer (Pewność: 17.27%)
--------------------------------------------------
Finalna predykcja Dzielnicy: Wawer (Pewność: 21.05%)
Finalna predykcja Ulicy:     Wojciecha gersona (Pewność: 18.27%)
